<a href="https://colab.research.google.com/github/isabelklint/scrapers/blob/main/Fanabc_WEB_SCRAPER_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [2]:
# https://github.com/tilburgsciencehub/data-ranking-the-brands/blob/main/webscraping_ranking_the_brands.ipynb
!pip install BeautifulSoup4
!pip install pandas
!pip install requests
!pip install cloudscraper

from bs4 import BeautifulSoup
import requests, lxml
import pandas as pd
import cloudscraper


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00


In [3]:
# make a list for urls that we assume are real
generated_urls = []

# use the format of the existing categories: 
categories = ['https://www.fanabc.com/archives/category/localnews/page/',
              'https://www.fanabc.com/archives/category/worldnews/page/',
              'https://www.fanabc.com/archives/category/sport/page/',
              'https://www.fanabc.com/archives/category/business/page/',
              'https://www.fanabc.com/archives/category/health/page/',
              'https://www.fanabc.com/archives/category/tech/page/']

# generate the urls
for category_url in categories:
    for i in range(1, 2):
        page_url = category_url + str(i)
        generated_urls.append(page_url)

# print(generated_urls)

In [4]:
len(generated_urls)

6

In [5]:
# target tags
# <h2 class="title">
#   <a href="https://www.fanabc.com/archives/188267" class="post-url post-title">
#           የክልል ልዩ ኃይልን መልሰን እናደራጀው እንጂ  እንበትነው አላልንም &#8211; ጄነራል አበባው ታደሰ
#   <span class="time">
#     <time class="post-published updated" datetime="2023-04-07T22:56:23+03:00">Apr 7, 2023</time>


In [6]:
data = pd.DataFrame(columns = ['Date', 'Title', 'URL'])


In [7]:
def scrape_this_url(url, unique_urls, max_retries=3):
    scraper = cloudscraper.create_scraper()
    headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
    
    data_list = []

    retries = 0
    while retries < max_retries:
        try:
            r = scraper.get(url, headers=headers, timeout=10)
            # create a BeautifulSoup object with the HTML code
            soup = BeautifulSoup(r.text, 'html.parser')

            articles = soup.find_all('article')

            for article in articles:
                title = article.find('h2', class_="title").text if article.find('h2', class_="title") else ""
                url = article.find('a')['href'] if article.find('a') else ""
                date = article.find("time", class_="post-published").get_text().strip() if article.find("time", class_="post-published") else ""
                print(title, date, url)

                data_list.append({'title': title, 'url': url, 'date': date})

            break  # exit the while loop if successful

        except Exception as e:
            retries += 1
            print(e)
            continue  # retry the request if unsuccessful

    # return the list of dictionaries
    return data_list


In [ ]:
import time

unique_urls = []
data_list = []

for url in generated_urls:
    print('start ', url)
    data_dict = scrape_this_url(url, unique_urls)
    if data_dict:
        data_list += data_dict
    time.sleep(2) # wait for 2 seconds before making the next request

# convert the list of dictionaries to a pandas DataFrame
data = pd.DataFrame(data_list)


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 82
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   38 non-null     object
 1   url     38 non-null     object
 2   date    38 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


In [11]:
data.drop_duplicates(subset=['url'], keep='first', inplace=True)


In [12]:
data.head()


,title,url,date
0,\n\t\t\t\tኢትዮጵያ ወንጀልን በመከላከል ለቀጣናው ሰላም እያከናወነ...,https://www.fanabc.com/archives/189213,"Apr 13, 2023"
1,\n\t\t\t\tበመዲናዋ መሬት አሰጥሃለሁ በማለት ከአንድ ግለሰብ 30 ...,https://www.fanabc.com/archives/189195,"Apr 13, 2023"
2,\n\t\t\t\tምእመኑ በጾም ወቅት ያሳየውን መረዳዳትና አብሮነት የዘወ...,https://www.fanabc.com/archives/189191,
3,\n\t\t\t\tኢትዮጵያ የሳይበር ደህንነት ተሞክሮዋን አካፈለች\t\t\t\n,https://www.fanabc.com/archives/189188,
4,\n\t\t\tበአዲስ አበባ ከ43 ሺህ ካሬ ሜትር በላይ ሕገ ወጥ መሬት ...,https://www.fanabc.com/archives/category/local...,"Apr 13, 2023"


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 82
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   38 non-null     object
 1   url     38 non-null     object
 2   date    38 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


In [14]:
data.describe()

,title,url,date
count,38,38,38
unique,38,38,19
top,\n\t\t\t\tኢትዮጵያ ወንጀልን በመከላከል ለቀጣናው ሰላም እያከናወነ...,https://www.fanabc.com/archives/189213,
freq,1,1,12


In [16]:
from google.colab import files
data.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>